In [1]:
from tqdm import tqdm
import matplotlib.pyplot as plt # For plotting
import numpy as np  
import os
from Bio import SeqIO         # To run Biopython
import pandas as pd           # To read input data
import multiprocessing        # To run FIMO in parallel
from functools import partial # For parallel computing

In [2]:
runserver = 0

if runserver==1:
    ppath='/opt/data/netZooPy/regPrior/'
elif runserver==0:
    ppath=''

In [3]:
iterlimit= -1

In [4]:
precomputed = 0
pathos=os.getcwd()
pathos

'/Users/anwer/Desktop/Gene_Regulatory_Network_ROSMAP/code'

In [19]:
def reduceSequence(sequence):
    seq=sequence[:2000]#since start is tss-1000, then we take the 2000bp upstream 
    return seq

In [7]:
geneCorr = pd.read_csv('./Regulatory_Prior_Network_Data/hg38_Tss_coordinates.csv',sep='\t')

In [8]:
geneCorr.head()

,#bin,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,score,name2,cdsStartStat,cdsEndStat,exonFrames
0,0,XM_011541469.1,chr1,-,67092175,67109072,67093004,67103382,5,"67092175,67095234,67096251,67103237,67109028,","67093604,67095421,67096321,67103382,67109072,",0,C1orf141,cmpl,cmpl,"0,2,1,0,-1,"
1,0,XM_011541467.1,chr1,-,67092175,67131183,67093004,67127240,9,"67092175,67095234,67096251,67103237,67111576,6...","67093604,67095421,67096321,67103343,67111644,6...",0,C1orf141,cmpl,cmpl,"0,2,1,0,1,2,0,0,-1,"
2,0,XM_017001276.1,chr1,-,67092175,67131227,67093004,67127240,9,"67092175,67095234,67096251,67103237,67111576,6...","67093604,67095421,67096321,67103382,67111644,6...",0,C1orf141,cmpl,cmpl,"0,2,1,0,1,2,0,0,-1,"
3,0,XM_011541465.2,chr1,-,67092175,67134962,67093004,67127240,9,"67092175,67095234,67096251,67103237,67111576,6...","67093604,67095421,67096321,67103382,67111644,6...",0,C1orf141,cmpl,cmpl,"0,2,1,0,1,2,0,0,-1,"
4,0,NR_075077.1,chr1,-,67092175,67134971,67134971,67134971,10,"67092175,67096251,67103237,67111576,67113613,6...","67093604,67096321,67103382,67111644,67113756,6...",0,C1orf141,none,none,"-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,"


In [159]:
geneCorr

,#bin,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,score,name2,cdsStartStat,cdsEndStat,exonFrames
0,0,XM_011541469.1,chr1,-,67092175,67109072,67093004,67103382,5,"67092175,67095234,67096251,67103237,67109028,","67093604,67095421,67096321,67103382,67109072,",0,C1orf141,cmpl,cmpl,"0,2,1,0,-1,"
1,0,XM_011541467.1,chr1,-,67092175,67131183,67093004,67127240,9,"67092175,67095234,67096251,67103237,67111576,6...","67093604,67095421,67096321,67103343,67111644,6...",0,C1orf141,cmpl,cmpl,"0,2,1,0,1,2,0,0,-1,"
2,0,XM_017001276.1,chr1,-,67092175,67131227,67093004,67127240,9,"67092175,67095234,67096251,67103237,67111576,6...","67093604,67095421,67096321,67103382,67111644,6...",0,C1orf141,cmpl,cmpl,"0,2,1,0,1,2,0,0,-1,"
3,0,XM_011541465.2,chr1,-,67092175,67134962,67093004,67127240,9,"67092175,67095234,67096251,67103237,67111576,6...","67093604,67095421,67096321,67103382,67111644,6...",0,C1orf141,cmpl,cmpl,"0,2,1,0,1,2,0,0,-1,"
4,0,NR_075077.1,chr1,-,67092175,67134971,67134971,67134971,10,"67092175,67096251,67103237,67111576,67113613,6...","67093604,67096321,67103382,67111644,67113756,6...",0,C1orf141,none,none,"-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167464,586,XM_017030167.1,chr22_KI270734v1_random,-,138085,153840,138479,152899,13,"138085,138742,142193,143613,144748,145003,1466...","138667,138831,142292,143789,144895,145096,1467...",0,LOC102724788,cmpl,cmpl,"1,2,2,0,0,0,2,0,0,0,1,1,0,"
167465,586,XR_951398.2,chr22_KI270734v1_random,-,138085,161588,161588,161588,14,"138085,138742,142193,143613,144748,145003,1466...","138667,138831,142292,143789,144929,145096,1467...",0,LOC102724788,none,none,"-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,"
167466,586,XM_017030168.1,chr22_KI270734v1_random,-,138085,161592,138479,156446,14,"138085,138742,142193,143613,144748,145003,1466...","138667,138831,142292,143789,144895,145096,1467...",0,LOC102724788,cmpl,cmpl,"1,2,2,0,0,0,2,0,0,1,1,2,0,-1,"
167467,586,XM_006724936.3,chr22_KI270734v1_random,-,138085,161594,138479,150995,13,"138085,138742,142193,143613,144748,145003,1466...","138667,138831,142292,143789,144895,145096,1467...",0,LOC102724788,cmpl,cmpl,"1,2,2,0,0,0,2,0,0,0,0,-1,-1,"


In [180]:
geneCorr[geneCorr.name2 == 'TSNAX-DISC1']

,#bin,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,score,name2,cdsStartStat,cdsEndStat,exonFrames
907,36,NR_028400.1,chr1,+,231528652,231750619,231750619,231750619,8,"231528652,231529254,231537212,231542480,231616...","231528826,231529359,231537327,231542611,231616...",0,TSNAX-DISC1,none,none,"-1,-1,-1,-1,-1,-1,-1,-1,"
908,36,NR_028398.1,chr1,+,231528652,231819244,231819244,231819244,11,"231528652,231529254,231561127,231693825,231701...","231528826,231529359,231561255,231694805,231702...",0,TSNAX-DISC1,none,none,"-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,"
909,36,NR_028396.1,chr1,+,231528652,231819244,231819244,231819244,13,"231528652,231529254,231537212,231542480,231616...","231528826,231529359,231537327,231542611,231616...",0,TSNAX-DISC1,none,none,"-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,"
910,36,NR_028397.1,chr1,+,231528652,231819244,231819244,231819244,13,"231528652,231529254,231537212,231542480,231561...","231528826,231529359,231537327,231542611,231561...",0,TSNAX-DISC1,none,none,"-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,"
911,36,NR_028395.1,chr1,+,231528652,231819244,231819244,231819244,13,"231528652,231529254,231537212,231542480,231561...","231528826,231529359,231537327,231542611,231561...",0,TSNAX-DISC1,none,none,"-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,"
912,36,NR_028394.1,chr1,+,231528652,231819244,231819244,231819244,14,"231528652,231529254,231537212,231542480,231561...","231528826,231529359,231537327,231542611,231561...",0,TSNAX-DISC1,none,none,"-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,"
913,36,NR_028393.1,chr1,+,231528652,232041272,232041272,232041272,16,"231528652,231529254,231537212,231542480,231616...","231528826,231529359,231537327,231542611,231616...",0,TSNAX-DISC1,none,none,"-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,"
5380,293,NR_028399.1,chr1,+,231528652,231702561,231702561,231702561,9,"231528652,231529254,231537212,231542480,231561...","231528826,231529359,231537327,231542611,231561...",0,TSNAX-DISC1,none,none,"-1,-1,-1,-1,-1,-1,-1,-1,-1,"


In [165]:
k=0 # iteration counter
input_file ='./Regulatory_Prior_Network_Data/hg38_sequence_Tss1000_Tse1000.fasta'
output_file='./Regulatory_Prior_Network_Data/output/hg38_sequence_Tss1000_Tss1000.fasta'

fasta_sequences = SeqIO.parse(open(input_file),'fasta')


In [179]:
l =0
finalSeq = ''
namelist = []
for fasta in tqdm(fasta_sequences):
    print(fasta)
    name, sequence = fasta.id, str(fasta.seq)
    new_sequence = reduceSequence(sequence)
    name = name[16:]
    print('name', name)
    boolInd = np.in1d(geneCorr.iloc[:,1],name)
    name = geneCorr.iloc[boolInd,12].values[0]
    interName = np.intersect1d(name, namelist)
    print(namelist)
    if interName.size == 0:
        
        namelist.append(name)
        
    else:
        print(name)
      
       
        continue
    name = '>' + name
    finalSeq = finalSeq + name + '\n' + new_sequence + '\n'
    
   
    
        

1it [00:00, 58.98it/s]

ID: hg38_ncbiRefSeq_NR_028394.1
Name: hg38_ncbiRefSeq_NR_028394.1
Description: hg38_ncbiRefSeq_NR_028394.1 range=chr1:231527653-231820244 5'pad=0 3'pad=0 strand=+ repeatMasking=none
Number of features: 0
Seq('acagcgctgagaagcactggggctaaggggcggcacaaagtctagatatggtgg...aca')
name NR_028394.1
[]
TSNAX-DISC1
ID: hg38_ncbiRefSeq_NR_028393.1
Name: hg38_ncbiRefSeq_NR_028393.1
Description: hg38_ncbiRefSeq_NR_028393.1 range=chr1:231527653-232042272 5'pad=0 3'pad=0 strand=+ repeatMasking=none
Number of features: 0
Seq('acagcgctgagaagcactggggctaaggggcggcacaaagtctagatatggtgg...agt')
name NR_028393.1
['TSNAX-DISC1']
TSNAX-DISC1


In [168]:
namelist

['WDR26']

In [150]:
finalSeq[0:2000]

'>C1orf141\naacagcatttatattattatttataaataatcatttcatttacattatagtaatgtctctgtaataatcattaccgaacaaagtaatgtgcttggattacctttcttctcagtgggtacaatttaacaacaatggactcttttttctctatctattactaaaaaaatgccatgttttagtttccctcatatttagaccttaactgttcagtacacttttatgttttttcttggactttctaatggcctttcttttttcttgtgggaaatgatgataataatatcaaacatgtaattgcatatgtgcctcaagcactattcaaagaaaaattacatcctcaacaacccaatgacgtaaatactattatccctgagaaaaaagaggcacaaggaagataagcaacttgcctgggatcaaatagtatatgacagaaccaggaagaacccagcagtttgattcatgctcattgtcactaagatctgcagcctccaaaaaagaagcatgtaactttcttatcatatgtgaaaatgatcaagttttttattcattctctctattgtgtgtaatccattccaatgttcttggaggaattgttcatgaagcccagtgacttcttgttctaaatttgacctgttacctataagcctacttttacattgtcattccattcctttgctcagttcctcagattctagtatcccatgtctccttattccttttttttttttttttttttttttttttttgagacggagtctcgctctgtcgcccaggctggagtgcagtggcgccatctcggctcactgcaagctctgcctcctgcgttcaggccattctcctgcctcagcctcccgagtagctgggactacaggcgtcagccaccatgctggctaatttttttgtatttttagtagagacggggtttcaccgtgttagccaggatggtctcgatctcccgacctcgtgatccgcctgcctcggcctcaca

In [149]:
if precomputed==0:
    # save file
    with open(output_file, 'w') as file:
        file.write(finalSeq)

In [158]:
os.chdir(pathos)
geneNames  = []
fasta_sequences = SeqIO.parse(open('./Regulatory_Prior_Network_Data/output/hg38_sequence_Tss1000_Tss1000.fasta'),'fasta')
for fasta in tqdm(fasta_sequences):
    x = 1
    

38723it [00:00, 152102.99it/s]


In [259]:
os.chdir(pathos)
print(pathos)
os.chdir('./Regulatory_Prior_Network_Data/PWMs')
for file in tqdm(os.listdir()):
    df=pd.read_csv(file,sep='\t')
    df=df.iloc[:,1:]
    os.chdir(pathos)
    os.chdir('./Regulatory_Prior_Network_Data/output/convPWMs')
    df.to_csv(file, header=False, index=False, sep='\t')
    os.chdir(pathos)

    os.chdir('./Regulatory_Prior_Network_Data/PWMs')
os.chdir(pathos)


/Users/anwer/Desktop/Gene_Regulatory_Network_ROSMAP/code


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5597/5597 [00:26<00:00, 213.65it/s]


In [260]:
os.chdir(pathos)
os.chdir('./Regulatory_Prior_Network_Data/output/convPWMs')
finalTfList = []


for file in tqdm(os.listdir()):
    bashCommand = "/Users/anwer/meme/libexec/meme-5.5.2/matrix2meme <" + file + "> " + file + ".meme"
    res=os.system(bashCommand)
    if res != 0:
        print(file)
    else:
        finalTfList.append(file[:-4])


  0%|                                                                                                                                                                                | 3/5599 [00:00<19:39,  4.75it/s]Error: Motif position 17 summed to zero.
Died at /Users/anwer/meme/libexec/meme-5.5.2/matrix2meme line 117, <STDIN> line 26.
  0%|▏                                                                                                                                                                               | 5/5599 [00:00<13:01,  7.16it/s]

M10303_1.94d.txt


 15%|█████████████████████████▉                                                                                                                                                    | 833/5599 [01:03<05:57, 13.34it/s]Error: Expected either the row or column count to be the alphabet size 4 but got 10 by 1.
Died at /Users/anwer/meme/libexec/meme-5.5.2/matrix2meme line 117, <STDIN> line 1.
 15%|██████████████████████████                                                                                                                                                    | 837/5599 [01:03<05:52, 13.51it/s]

.DS_Store


 18%|███████████████████████████████                                                                                                                                               | 999/5599 [01:15<06:01, 12.71it/s]Error: Motif position 8 summed to zero.
Died at /Users/anwer/meme/libexec/meme-5.5.2/matrix2meme line 117, <STDIN> line 19.
 18%|██████████████████████████████▉                                                                                                                                              | 1003/5599 [01:16<05:51, 13.09it/s]

...

 86%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                       | 4839/5599 [06:15<00:58, 13.04it/s]Error: Motif position 12 summed to zero.
Died at /Users/anwer/meme/libexec/meme-5.5.2/matrix2meme line 117, <STDIN> line 28.
 86%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                       | 4843/5599 [06:16<00:58, 13.01it/s]

M10282_1.94d.txt


 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎ | 5546/5599 [07:12<00:04, 11.78it/s]Error: Motif position 15 summed to zero.
Died at /Users/anwer/meme/libexec/meme-5.5.2/matrix2meme line 117, <STDIN> line 25.
 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍ | 5550/5599 [07:12<00:03, 12.36it/s]

M10278_1.94d.txt


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5599/5599 [07:16<00:00, 12.83it/s]


In [261]:
os.chdir(pathos)

In [262]:
os.chdir(pathos)

tf = pd.read_csv('./Regulatory_Prior_Network_Data/Human_TF_MotifList_v_1.01.csv',dtype = str)
tf

,Ensembl ID,HGNC symbol,Motif evidence,Motif ID,Motif type,Motif source,CIS-BP ID,Best Motif(s)? (Figure 2A)
0,ENSG00000267281,AC023509.3,"Inferred - ATF7 (100% AA Identity, Homo sapiens)",ATF7_eDBD_HT-SELEX,HT-SELEX,Yin2017,M08490_1.94d,TRUE
1,ENSG00000267281,AC023509.3,"Inferred - ATF7 (100% AA Identity, Homo sapiens)",ATF7_eDBD_Methyl-HT-SELEX,Methyl-HT-SELEX,Yin2017,M08491_1.94d,NaN
2,ENSG00000267281,AC023509.3,"Inferred - Atf7 (98% AA Identity, Mus musculus)",3T3L1-Atf7_GSE56872,Misc,HOMER,M06975_1.94d,NaN
3,ENSG00000264668,AC138696.1,"Inferred - ZFP41 (100% AA Identity, Homo sapiens)",ZFP41_FL_HT-SELEX,HT-SELEX,Yin2017,M09315_1.94d,TRUE
4,ENSG00000264668,AC138696.1,"Inferred - ZFP41 (100% AA Identity, Homo sapiens)",ZFP41_FL_Methyl-HT-SELEX,Methyl-HT-SELEX,Yin2017,M09316_1.94d,NaN
...,...,...,...,...,...,...,...,...
26985,ENSG00000204532,ZSCAN5C,"Inferred - ZSCAN5 (82% AA Identity, Homo sapiens)",ZSCAN5_eDBD_HT-SELEX,HT-SELEX,Yin2017,M09419_1.94d,NaN
26986,ENSG00000204532,ZSCAN5C,"Inferred - ZSCAN5 (82% AA Identity, Homo sapiens)",ZSCAN5_eDBD_Methyl-HT-SELEX,Methyl-HT-SELEX,Yin2017,M09420_1.94d,NaN
26987,ENSG00000137185,ZSCAN9,Direct,ZSCAN9_eDBD_HT-SELEX,HT-SELEX,Yin2017,M09421_1.94d,TRUE
26988,ENSG00000137185,ZSCAN9,Direct,ZSCAN9_eDBD_Methyl-HT-SELEX,Methyl-HT-SELEX,Yin2017,M09422_1.94d,NaN


In [263]:
#tf = pd.read_csv('./Regulatory_Prior_Network_Data/TF_Information_all_motifs_plus.txt',sep='\t')
#tf

In [264]:
tf.columns

Index(['Ensembl ID', 'HGNC symbol', 'Motif evidence', 'Motif ID', 'Motif type',
       'Motif source', 'CIS-BP ID', 'Best Motif(s)? (Figure 2A)'],
      dtype='object')

In [265]:
'''indTF = np.in1d(tf.iloc[:,3],finalTfList)
tff = tf
tf = tf.iloc[indTF,:]
initTF = tf.iloc[0,1]'''

'indTF = np.in1d(tf.iloc[:,3],finalTfList)\ntff = tf\ntf = tf.iloc[indTF,:]\ninitTF = tf.iloc[0,1]'

In [266]:
'''tflist, tfflist = [], []
tflist.extend(tf.iloc[:,3])
tfflist.extend(tf.iloc[:,6])
tfFound = 0 '''

'tflist, tfflist = [], []\ntflist.extend(tf.iloc[:,3])\ntfflist.extend(tf.iloc[:,6])\ntfFound = 0 '

In [267]:
precomputed == 0
if precomputed==0:
    # read TF motif table to select the "best" motif per TF
    indTF  =np.in1d(tf.iloc[:,6],finalTfList)
    tff    =tf
    tf     =tf.iloc[indTF,:]
    initTF =tf.iloc[0,1]
    tflist, tfflist =[],[]
    tflist.append(tf.iloc[0,6])
    tfflist.append(initTF)
    tfFound=0
    for i in range(tf.shape[0]):
        newTF = tf.iloc[i, 1]
        if tfFound==1 and initTF==newTF:
            continue
        else:
            if initTF!=newTF:
                if initTF != newTF and tfFound==0:
                    tflist.append(tf.iloc[i, 6])
                    tfflist.append(tf.iloc[i, 1])
                tfFound=0
                initTF =newTF
            if tf.iloc[i,7]==True:
                tflist.append(tf.iloc[i,6])
                tfflist.append(tf.iloc[i, 1])
                tfFound=1

In [294]:
print(tflist[0:10])
print(tfflist[0:10])

['M08490_1.94d', 'M09315_1.94d', 'M06439_1.94d', 'M06439_1.94d', 'M06440_1.94d', 'M08464_1.94d', 'M05559_1.94d', 'M05562_1.94d', 'M01298_1.94d', 'M06442_1.94d']
['AC023509.3', 'AC138696.1', 'AHR', 'AHRR', 'AIRE', 'ALX1', 'ALX3', 'ALX4', 'ANHX', 'AR']


In [301]:
len(tfflist)

1149

In [300]:
os.chdir(pathos)
import re
if precomputed==0:
    # put pwms in the same file
    os.chdir('./Regulatory_Prior_Network_Data/output/convPWMs')
    finalMeme = 'MEME version 5.5.2\n\nALPHABET= ACGT\n\nstrands: + -\n\nBackground letter frequencies (from uniform background):\nA 0.25000 C 0.25000 G 0.25000 T 0.25000 \n\n'
    k=0
    finalTFName=[]
    for i,j in tqdm(enumerate(range(len(tflist)))):
        try:
            file = open(tflist[i] + '.txt.meme', 'r')
            k    = k+1
            finalTFName.append(tfflist[i])
            data = file.read()
            start = re.search("letter-probability", data).start()
            finalMeme = finalMeme +'MOTIF ' + tfflist[i] + data[start-1:]
            
            file.close()
            
        except:
            print("TF not found")
    print(finalMeme)
    # save file
    os.chdir(pathos)
    os.chdir('./Regulatory_Prior_Network_Data/output/convPWMs')
    with open('allTFs.meme', 'w') as file:
        file.write(finalMeme)

    # save TFs
    with open("tfNames.txt", "w") as f:
        for s in finalTFName:
            f.write(str(s) +"\n")


1149it [00:00, 5146.25it/s]

In [236]:
'''os.chdir(pathos)
os.chdir('./Regulatory_Prior_Network_Data/output/convPWMs')


with open('allTFs.meme', 'w') as file:
    file.write(finalMeme)

    # save TFs
with open("tfNames.txt", "w") as f:
    for s in finalTFName:
        f.write(str(s) +"\n")'''


'os.chdir(pathos)\nos.chdir(\'./Regulatory_Prior_Network_Data/output/convPWMs\')\n\n\nwith open(\'allTFs.meme\', \'w\') as file:\n    file.write(finalMeme)\n\n    # save TFs\nwith open("tfNames.txt", "w") as f:\n    for s in finalTFName:\n        f.write(str(s) +"\n")'

In [302]:
os.chdir(pathos)
os.chdir('./Regulatory_Prior_Network_Data/output/convPWMs')
tfNames=[]

with open("tfNames.txt", "r") as f:
  for line in f:
    tfNames.append(str(line.strip()))

In [303]:
len(tfNames)

1149

In [304]:
os.chdir(pathos)
geneNames  = []
fasta_sequences = SeqIO.parse(open('./Regulatory_Prior_Network_Data/output/hg38_sequence_Tss1000_Tss1000.fasta'),'fasta')
for fasta in tqdm(fasta_sequences):
        name, sequence = fasta.id, str(fasta.seq)
        geneNames.append(name)

nTFs       = len(tfNames)


38723it [00:00, 131251.01it/s]


In [240]:
print(len(geneNames))
os.chdir(pathos)


38723


In [306]:
tf.head()

,Ensembl ID,HGNC symbol,Motif evidence,Motif ID,Motif type,Motif source,CIS-BP ID,Best Motif(s)? (Figure 2A)
0,ENSG00000267281,AC023509.3,"Inferred - ATF7 (100% AA Identity, Homo sapiens)",ATF7_eDBD_HT-SELEX,HT-SELEX,Yin2017,M08490_1.94d,TRUE
1,ENSG00000267281,AC023509.3,"Inferred - ATF7 (100% AA Identity, Homo sapiens)",ATF7_eDBD_Methyl-HT-SELEX,Methyl-HT-SELEX,Yin2017,M08491_1.94d,NaN
2,ENSG00000267281,AC023509.3,"Inferred - Atf7 (98% AA Identity, Mus musculus)",3T3L1-Atf7_GSE56872,Misc,HOMER,M06975_1.94d,NaN
3,ENSG00000264668,AC138696.1,"Inferred - ZFP41 (100% AA Identity, Homo sapiens)",ZFP41_FL_HT-SELEX,HT-SELEX,Yin2017,M09315_1.94d,TRUE
4,ENSG00000264668,AC138696.1,"Inferred - ZFP41 (100% AA Identity, Homo sapiens)",ZFP41_FL_Methyl-HT-SELEX,Methyl-HT-SELEX,Yin2017,M09316_1.94d,NaN


In [310]:
tfNames[0]

'AC023509.3'

In [351]:
def FIMO(tfi, geneNames, tfNames, pqval):
    os.chdir(pathos)
    os.chdir('./Regulatory_Prior_Network_Data/output/fimo_output/')
    regMatQval = np.zeros((1,len(geneNames))) 
    regMatQ = pd.DataFrame(data = regMatQval, columns = geneNames, index = [tfNames[tfi]])
    
    bashCommand = '/Users/anwer/meme/bin/fimo --thresh 1e-5 --no-qvalue --verbosity 1 --max-stored-scores 100000000 --oc '+tfNames[tfi]+' --motif ' + tfNames[tfi] + ' /Users/anwer/Desktop/Gene_Regulatory_Network_ROSMAP/code/Regulatory_Prior_Network_Data/output/ConvPWMs/allTFs.meme  /Users/anwer/Desktop/Gene_Regulatory_Network_ROSMAP/code/Regulatory_Prior_Network_Data/output/hg38_sequence_Tss1000_Tss1000.fasta'
    res = os.system(bashCommand)
    if res != 0:
        print('could not find fimo')
    
    else:
        
        os.chdir(pathos)
        os.chdir('./Regulatory_Prior_Network_Data/output/fimo_output/' + tfNames[tfi])
        
    try:
        tf = pd.read_csv('fimo.tsv',sep='\t', comment='#')
        tf.iloc[:,3] = 1000 - tf.iloc[:,3]
        for geneName in geneNames:
            jset = np.where(np.in1d(tf.iloc[:,2],geneName))
            if jset[0].size != 0:
                regMatQ[geneName].iloc[0] = tf.iloc[jset[0],7].min()
                
    except: 
        print('empty fimo result')
    
    return regMatQ

In [ ]:
for tfi in range(0,1):
    
    res = FIMO(tfi, geneNames, tfNames, pqval = 0 )
   
for tfi in tqdm(range(1,nTFs)):
     res  = pd.concat([res, FIMO(tfi, geneNames, tfNames, pqval = 0 )])   
os.chdir(pathos)
    



  0%|                                                                                                                                                                                        | 0/1148 [00:00<?, ?it/s]

  0%|▏                                                                                                                                                                             | 1/1148 [00:22<7:15:46, 22.80s/it]

  0%|▎                                                                                                                                                                             | 2/1148 [00:42<6:42:36, 21.08s/it]

  0%|▍                                                                                                                                                                             | 3/1148 [01:02<6:28:08, 20.34s/it]

  0%|▌                                                                                                                                

empty fimo result




  1%|█▎                                                                                                                                                                            | 9/1148 [03:08<6:37:54, 20.96s/it]

  1%|█▌                                                                                                                                                                           | 10/1148 [03:29<6:34:46, 20.81s/it]

  1%|█▋                                                                                                                                                                           | 11/1148 [04:00<7:31:04, 23.80s/it]

  1%|█▊                                                                                                                                                                           | 12/1148 [04:20<7:11:15, 22.78s/it]

  1%|█▉                                                                                                                               

empty fimo result




  3%|██████                                                                                                                                                                       | 40/1148 [14:38<6:34:37, 21.37s/it]

  4%|██████▏                                                                                                                                                                      | 41/1148 [15:01<6:47:50, 22.11s/it]

  4%|██████▎                                                                                                                                                                      | 42/1148 [15:24<6:48:37, 22.17s/it]

  4%|██████▍                                                                                                                                                                      | 43/1148 [15:45<6:41:29, 21.80s/it]

  4%|██████▋                                                                                                                          

...



  5%|████████▍                                                                                                                                                                    | 56/1148 [20:41<6:51:07, 22.59s/it]

  5%|████████▌                                                                                                                                                                    | 57/1148 [20:59<6:29:36, 21.43s/it]

empty fimo result




  5%|████████▋                                                                                                                                                                    | 58/1148 [21:21<6:29:25, 21.44s/it]

  5%|████████▉                                                                                                                                                                    | 59/1148 [21:44<6:37:09, 21.88s/it]

  5%|█████████                                                                                                                                                                    | 60/1148 [22:05<6:31:27, 21.59s/it]

  5%|█████████▏                                                                                                                                                                   | 61/1148 [22:26<6:28:03, 21.42s/it]

  5%|█████████▎                                                                                                                       

empty fimo result




  6%|███████████                                                                                                                                                                  | 73/1148 [26:40<6:19:44, 21.19s/it]

  6%|███████████▏                                                                                                                                                                 | 74/1148 [27:02<6:27:02, 21.62s/it]

  7%|███████████▎                                                                                                                                                                 | 75/1148 [27:25<6:31:41, 21.90s/it]

  7%|███████████▍                                                                                                                                                                 | 76/1148 [27:46<6:29:55, 21.82s/it]

  7%|███████████▌                                                                                                                     

In [321]:
for geneName in geneNames:
    print(geneName)
    print(tf.iloc[:,2])
    jset = np.where(np.in1d(tf.iloc[:,2],'ATF7'))
    break

C1orf141
0         Inferred - ATF7 (100% AA Identity, Homo sapiens)
1         Inferred - ATF7 (100% AA Identity, Homo sapiens)
2          Inferred - Atf7 (98% AA Identity, Mus musculus)
3        Inferred - ZFP41 (100% AA Identity, Homo sapiens)
4        Inferred - ZFP41 (100% AA Identity, Homo sapiens)
                               ...                        
26984                                               Direct
26985    Inferred - ZSCAN5 (82% AA Identity, Homo sapiens)
26986    Inferred - ZSCAN5 (82% AA Identity, Homo sapiens)
26987                                               Direct
26988                                               Direct
Name: Motif evidence, Length: 22925, dtype: object


In [337]:
print(len(geneNames))

38723


In [323]:
tf

,Ensembl ID,HGNC symbol,Motif evidence,Motif ID,Motif type,Motif source,CIS-BP ID,Best Motif(s)? (Figure 2A)
0,ENSG00000267281,AC023509.3,"Inferred - ATF7 (100% AA Identity, Homo sapiens)",ATF7_eDBD_HT-SELEX,HT-SELEX,Yin2017,M08490_1.94d,TRUE
1,ENSG00000267281,AC023509.3,"Inferred - ATF7 (100% AA Identity, Homo sapiens)",ATF7_eDBD_Methyl-HT-SELEX,Methyl-HT-SELEX,Yin2017,M08491_1.94d,NaN
2,ENSG00000267281,AC023509.3,"Inferred - Atf7 (98% AA Identity, Mus musculus)",3T3L1-Atf7_GSE56872,Misc,HOMER,M06975_1.94d,NaN
3,ENSG00000264668,AC138696.1,"Inferred - ZFP41 (100% AA Identity, Homo sapiens)",ZFP41_FL_HT-SELEX,HT-SELEX,Yin2017,M09315_1.94d,TRUE
4,ENSG00000264668,AC138696.1,"Inferred - ZFP41 (100% AA Identity, Homo sapiens)",ZFP41_FL_Methyl-HT-SELEX,Methyl-HT-SELEX,Yin2017,M09316_1.94d,NaN
...,...,...,...,...,...,...,...,...
26984,ENSG00000204532,ZSCAN5C,Direct,A6NGD5_1-5.RCADE,RCADE,Schmitges2016,M10321_1.94d,TRUE
26985,ENSG00000204532,ZSCAN5C,"Inferred - ZSCAN5 (82% AA Identity, Homo sapiens)",ZSCAN5_eDBD_HT-SELEX,HT-SELEX,Yin2017,M09419_1.94d,NaN
26986,ENSG00000204532,ZSCAN5C,"Inferred - ZSCAN5 (82% AA Identity, Homo sapiens)",ZSCAN5_eDBD_Methyl-HT-SELEX,Methyl-HT-SELEX,Yin2017,M09420_1.94d,NaN
26987,ENSG00000137185,ZSCAN9,Direct,ZSCAN9_eDBD_HT-SELEX,HT-SELEX,Yin2017,M09421_1.94d,TRUE


In [324]:
np.zeros((1,len(geneNames))) 

array([[0., 0., 0., ..., 0., 0., 0.]])